In [336]:
# This reloads the extension if already loaded - Everytime you update the .py files, re-run this cell!
%reload_ext autoreload
# Automatically reloads modules before executing code OR makes Jupyter reload your .py files whenever you run a cell.
%autoreload 2

In [338]:
# Importing all of the required python packages
import sys
import os

In [340]:
# Setting up the folder paths.
sys.path.append('./lib')

# Import resuable code (scripts) within .py files.
from data_loader import load_csv, save_df_to_csv
from data_wrangle import add_cols, df_formater, remove_cols, remove_rows, col_name_changer, df_split

Section: Loading and converting data sets into DataFrames.

In [343]:
df_indicators = load_csv('./data/raw/U.S._Chronic_Disease_Indicators.csv')
# print(df.head())

In [345]:
# df2 = load_csv('./data/raw/US_Census_Data_2022_v01.csv')
# print(df2.head())

In [347]:
df_census = load_csv('./data/raw/US_Census_Data_2022_v03_transpose.csv')
# display(df3.head())

Section: Data Wrangling

Census DataFrame: df_census
1. Adding the state column to the dataframe.
2. Adding the correct state name to the newly created state column.
3. Removing blank columns
4. Removing blank rows
5. Split dataset into 2, 1 with state and other is city, state
6. Saves both datasets into 2 csv files at filepath: data/processed/

In [351]:
col_names = ['State']
df_census_temp0 = add_cols(df_census, col_names)
# display(df_census_temp0)    # For debugging only - comment out when not needed.

In [353]:
df_census_temp1 = df_formater(df_census_temp0)
# display(df_census_temp1)    # For debugging only - comment out when not needed.

In [355]:
df_census_temp2 = remove_cols(df_census_temp1)
# display(df_census_temp2)    # For debugging only - comment out when not needed.

In [357]:
df_census_temp3 = remove_rows(df_census_temp2)
# display(df_census_temp3)    # For debugging only - comment out when not needed.

In [359]:
og_string = "!!"
new_string = " -- "
df_census_temp4 = col_name_changer(df_census_temp3, og_string, new_string)
# display(df_census_temp4)     # For debugging only - comment out when not needed.

In [361]:
df_state_only, df_state_city = df_split(df_census_temp4)
# display(df_state_only)    # For debugging only - comment out when not needed.
# display(df_state_city)    # For debugging only - comment out when not needed.

In [365]:
save_df_to_csv(df_state_only, './data/processed/US_Census_Data_2022_state_only.csv')

Dataframe:              State Label (Grouping)  \
0          Alabama         Estimate   
1          Alabama  Margin of Error   
2           Alaska         Estimate   
3           Alaska  Margin of Error   
4          Arizona         Estimate   
..             ...              ...   
97   West Virginia  Margin of Error   
98       Wisconsin         Estimate   
99       Wisconsin  Margin of Error   
100        Wyoming         Estimate   
101        Wyoming  Margin of Error   

    TOTAL NUMBER OF RACES REPORTED -- Total population  \
0                                            5,074,296   
1                                                *****   
2                                              733,583   
3                                                *****   
4                                            7,359,197   
..                                                 ...   
97                                               *****   
98                                           5,892,539  

In [367]:
save_df_to_csv(df_state_city, './data/processed/US_Census_Data_2022_city_state.csv')

Dataframe:                          State Label (Grouping)  \
0    Jefferson County, Alabama         Estimate   
1    Jefferson County, Alabama  Margin of Error   
2     Maricopa County, Arizona         Estimate   
3     Maricopa County, Arizona  Margin of Error   
4         Pima County, Arizona         Estimate   
..                         ...              ...   
355    San Antonio city, Texas  Margin of Error   
356   Seattle city, Washington         Estimate   
357   Seattle city, Washington  Margin of Error   
358  Milwaukee city, Wisconsin         Estimate   
359  Milwaukee city, Wisconsin  Margin of Error   

    TOTAL NUMBER OF RACES REPORTED -- Total population  \
0                                              665,409   
1                                                *****   
2                                            4,551,524   
3                                                *****   
4                                            1,057,597   
..                          